# 部署 TVM

In [3]:
from pathlib import Path

In [15]:
curr_path = Path(".").resolve()
base_path = curr_path/"lib"

In [20]:
import tvm
import numpy as np
from tvm import te
from tvm import relay


def prepare_test_libs(base_path):
    n = te.var("n")
    A = te.placeholder((n,), name="A")
    B = te.compute(A.shape, lambda *i: A(*i) + 1.0, name="B")
    s = te.create_schedule(B.op)
    # Compile library as dynamic library
    fadd_dylib = tvm.build(s, [A, B], "llvm", name="addone")
    dylib_path = str(base_path/"test_addone_dll.so")
    fadd_dylib.export_library(dylib_path)

    # Compile library in system library mode
    fadd_syslib = tvm.build(s, [A, B], "llvm --system-lib", name="addonesys")
    syslib_path = str(base_path/"test_addone_sys.o")
    fadd_syslib.save(syslib_path)


def prepare_graph_lib(base_path):
    x = relay.var("x", shape=(2, 2), dtype="float32")
    y = relay.var("y", shape=(2, 2), dtype="float32")
    params = {"y": np.ones((2, 2), dtype="float32")}
    mod = tvm.IRModule.from_expr(relay.Function([x, y], x + y))
    # build a module
    compiled_lib = relay.build(mod, tvm.target.Target("llvm"), params=params)
    # export it as a shared library
    # If you are running cross compilation, you can also consider export
    # to tar and invoke host compiler later.
    dylib_path = str(base_path/"test_relay_add.so")
    compiled_lib.export_library(dylib_path)

In [24]:
def verify(mod, fname):
    # Get the function from the module
    f = mod.get_function(fname)
    # Use tvm.nd.array to convert numpy ndarray to tvm
    # NDArray type, so that function can be invoked normally
    N = 10
    x = tvm.nd.array(np.arange(N, dtype=np.float32))
    y = tvm.nd.array(np.zeros(N, dtype=np.float32))
    # Invoke the function
    f(x, y)
    np_x = x.numpy()
    np_y = y.numpy()
    # Verify correctness of function
    assert np.all([xi + 1 == yi for xi, yi in zip(np_x, np_y)])
    print("Finish verification...")

In [25]:
# The normal dynamic loading method for deployment
mod_dylib = tvm.runtime.load_module("lib/test_addone_dll.so")
print("Verify dynamic loading from test_addone_dll.so")
verify(mod_dylib, "addone")

Verify dynamic loading from test_addone_dll.so
Finish verification...


In [27]:
mod_dylib

Module(library, 55fb48ef5148)